In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, re
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from gensim.models.doc2vec import TaggedDocument
import gensim.models.doc2vec
import pickle
from gensim.models import Doc2Vec
from fuzzywuzzy import fuzz
import types
from six import string_types, integer_types

C:\Users\Hesham\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Hesham\AppData\Local\Programs\Python\Python36\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data = pd.read_csv('./data_clean.csv')

In [3]:
with open('./docs_proc.pkl', 'rb') as file:
    docs = pickle.load(file)
    
print(len(docs))

37092


In [4]:
with open('all.json', 'w') as file:
    file.write('{"clean_sites" : [')
    file.write(','.join(['"'+data[data.url==doc.tags[0]].url.values[0]+'"' for doc in docs]))
    file.write(']}')

In [26]:
freq = []
for word, vocab_obj in model.wv.vocab.items():
    freq.append((str(word), vocab_obj.count))
sorted(freq, key=lambda x: x[1], reverse=True)

[('new', 54192),
 ('use', 44320),
 ('accessori', 42821),
 ('custom', 41446),
 ('design', 40855),
 ('view', 33280),
 ('massag_parlour', 30266),
 ('item', 29357),
 ('gift', 27085),
 ('one', 26539),
 ('make', 25669),
 ('collect', 25477),
 ('part', 25279),
 ('servic', 22513),
 ('set', 22406),
 ('kit', 22195),
 ('work', 22181),
 ('buy', 21268),
 ('inform', 21021),
 ('offer', 20252),
 ('see', 20229),
 ('help', 19986),
 ('best', 19789),
 ('special', 19742),
 ('avail', 19706),
 ('get', 19658),
 ('includ', 19625),
 ('time', 19483),
 ('find', 18630),
 ('ship', 18111),
 ('need', 18038),
 ('art', 17598),
 ('also', 17580),
 ('featur', 16943),
 ('size', 16855),
 ('color', 16730),
 ('free', 16668),
 ('style', 16415),
 ('light', 16358),
 ('stock', 16355),
 ('store', 16210),
 ('year', 16079),
 ('select', 15914),
 ('great', 15593),
 ('day', 15469),
 ('read', 15153),
 ('made', 15111),
 ('person', 14852),
 ('look', 14723),
 ('box', 14675),
 ('brand', 14590),
 ('top', 14575),
 ('book', 14528),
 ('compani',

In [42]:
display(docs[0])

TaggedDocument(words=['retro', 'planet', 'tin_sign', 'retro', 'decor', 'diner_furnitur', 'vintag', 'sign', 'uniqu_gift_idea', 'free_ship_tax', 'free', 'login', 'wish_list', 'view_cart_checkout', 'blog', 'cool', 'retro', 'gift', 'decor', 'item', 'wall_decor', 'metal', 'sign', 'wall_decal', 'floor', 'decal', 'clock_thermomet', 'sign', 'bundl', 'tryptych', 'neon_sign', 'poster_print', 'wood', 'sign', 'canva_wall_art', 'film_cell', 'gold', 'record', 'sign', 'decal', 'set', 'gift', 'sticker', 'mileston_birthday', 'gift_set', 'gift_basket', 'gift_wrap', 'tape', 'greet_card', 'tag', 'joke', 'gag_gift', 'toy', 'hero', 'geek', 'techi', 'pet_pet_lover', 'keychain', 'button_pin', 'kitchen_dine', 'bar', 'coffe_mug', 'tiki_mug', 'deli', 'serv', 'basket', 'kitchen_linen', 'kitchen_canist', 'jar', 'salt_pepper_shaker', 'glasswar', 'dispens', 'restaurantwar', 'tablewar_kitchen', 'drinkwar', 'bar', 'loung', 'parti', 'home_decor', 'funki', 'stuff', 'doormat', 'refriger_magnet', 'woven_throw', 'blanket',

# Doc2Vec

In [71]:
model = Doc2Vec(docs, vector_size = 500, window = 9, min_count = 20, workers=8, dm=0)

In [66]:
with open('./d2v_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [72]:
epochs = 5
start_alpha = 0.025
end_alpha = 0.0001
alpha_change = (start_alpha-end_alpha)/epochs
assert gensim.models.doc2vec.FAST_VERSION > -1
for i in range(epochs):
    print('Epoch', i)
    model.train(docs, total_examples=model.corpus_count, epochs=1)
    model.alpha -= alpha_change
    model.min_alpha = model.alpha

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [67]:
with open('./d2v_model_trained.pkl', 'wb') as file:
    pickle.dump(model, file)

In [94]:
# with open('./d2v_model_trained.pkl', 'rb') as file:
#     model = pickle.load(file)
#     print(model.vector_size, model.window, model.min_count)

In [75]:
strictness = 0.6
max_depth = 3
limit = 16
minimum = 2

def test_model(url):
#     print('Category:', data.label[data.url == url].values[0])
    n = 1
    ms = model.docvecs.most_similar(url, topn=n)
    while (ms[-1][1] > strictness and n < limit < len(docs)) or n < minimum:
        n += 1
        ms = model.docvecs.most_similar(url, topn=n)
    kw = []
    for entry in ms:
        if '.' in entry[0]:
            row = data[data.url == entry[0]]
            if row.empty:
                row = data[data.label.str.contains(entry[0])]
            print('(%.2f%% Match) URL: %s Category: %s'%(entry[1]*100, row.url.values[0], row.label.values[0]))
        else:
            kw.append(entry)
    print('Keywords: ' + ','.join([k[0]+'|'+ str(k[1]) for k in kw]))

    
def get_label(url):
    labels = []
    ms = model.docvecs.most_similar(url, topn=10)
    for m in ms:
        if '.' not in m[0]:
            labels.append(m)
    return labels

def get_recommended(url):
    results = dict()
    depth = 0
    keys = [url]
    n=1
    
    results = dict()
    while True:
        m = model.docvecs.most_similar(keys, topn=n)[-1]
        if '.' in m[0]:
            if not m[0] in results.keys():
                results[m[0]] = m[1]
            else:
                results[m[0]] = 2 / (1/results[m[0]] + 1/m[1])
            if len(results.items()) >= limit or (m[1] < strictness and not len(results) < minimum):
                break
        elif depth < max_depth:
            keys.append(m[0])
            depth += 1
            n=0
        n += 1
    
    results = sorted(results.items(), key=lambda x:x[1], reverse=True)
    
    filtered = [m for m in results if m[1] > strictness and '.' in m[0]]
    
    if(len(filtered) < minimum):
        filtered = results[:minimum]
    elif len(filtered) > limit:
        filtered = results[:limit]
    return filtered
    

In [76]:
urls = ['owners.com']
for url in urls:
    print(data[data.url == url].label)
    r = get_recommended(url)
    for m in r:
        url = m[0]
        cat = data[data.url == url].label.values[0]
        print(url, cat)

33444    Tobacco/Accessories
Name: label, dtype: object
eenee.com Children/Baby/Diapering
americase.com Recreation/Guns/Accessories/Gun Leather and Cases
gaytanfoods.com Food/Snacks/Pork Rinds
dealfarm.com Health/Beauty/Bath and Body/Soap/Handmade/Goat Milk
equilox.com rts/Equestrian/Healthcare and Grooming/Farrier and Hoof Care
gmansreptiles.com Pets/Reptiles and Amphibians
robertglennketchum.com Visual Arts/Photography/Photographers/Nature and Wildlife
nology.com Vehicles/Parts and Accessories/Performance
terrafurnishings.com Home and Garden/Furniture/Tables
jackmangallery.com.au Visual Arts/Ethnic and Regional/Oceanian/Australian
jollytime.com Food/Snacks/Popcorn
artlandmark.com Visual Arts/Posters and Prints/Prints/Gicl�e
cera.lt Antiques and Collectibles/Coins/Medals and Tokens
finetunegolf.com rts/Golf/Training Aids
realbesttest.com Visual Arts/Supplies/Graphic Arts and Illustration
magic25filter.com Tobacco/Accessories


In [78]:
test_model(['owners.com'])

(65.21% Match) URL: magic25filter.com Category: Tobacco/Accessories
(64.48% Match) URL: eenee.com Category: Children/Baby/Diapering
(64.12% Match) URL: realbesttest.com Category: Visual Arts/Supplies/Graphic Arts and Illustration
(61.83% Match) URL: stuffyourown.com Category: Tobacco/Cigarettes/Roll Your Own
(61.02% Match) URL: jollytime.com Category: Food/Snacks/Popcorn
(60.98% Match) URL: gmansreptiles.com Category: Pets/Reptiles and Amphibians
(60.98% Match) URL: jacksoldsouth.com Category: Food/Condiments/Sauces/Barbecue
Keywords: swisstray|0.972205638885498,amp_amp_ampamp_amp|0.6485229134559631,top_width_height|0.6446513533592224,gman|0.617594838142395,pop_corn|0.6166234016418457,homeproductssdsabout|0.6156175136566162,suvenyrai|0.608010470867157,rakt|0.6039752960205078,gaytan|0.6025441884994507


In [98]:
data[data.label == 'rts/Golf/Accessories/Cart']

,url,label,http,text
31997,buggiesunlimited.com,rts/Golf/Accessories/Cart,http://buggiesunlimited.com,Golf Cart Parts and Accessories | BuggiesUnlim...
31998,golfcarcatalog.com,rts/Golf/Accessories/Cart,http://golfcarcatalog.com,"Golf Cart Parts, Accessories and Golf Carts | ..."
31999,vintagegolfcartparts.com,rts/Golf/Accessories/Cart,http://vintagegolfcartparts.com,Home - Vintage Golf Cart Parts Inc. HomeShoppi...
32000,diygolfcart.com,rts/Golf/Accessories/Cart,http://diygolfcart.com,DIY Golf Cart Parts & Maintenance for Club Car...
32001,clubclean.net,rts/Golf/Accessories/Cart,http://clubclean.net,Club Clean | The Best in Golf Car Accessories ...
32002,directparts.com,rts/Golf/Accessories/Cart,http://directparts.com,Home page . to utilize the functionality of t...
32003,jasonsgolfcarts.com,rts/Golf/Accessories/Cart,http://jasonsgolfcarts.com,Buy Golf Cart Parts & Accessories Online | Hig...
32004,blockbustergolfcarts.com,rts/Golf/Accessories/Cart,http://blockbustergolfcarts.com,Golf Cart Replacement Parts | BlockbusterGolfC...
32005,parts4carts.com,rts/Golf/Accessories/Cart,http://parts4carts.com,"PARTS4CARTS, Parts4Carts.com - Everything you ..."
32006,cartbarn.com,rts/Golf/Accessories/Cart,http://cartbarn.com,Golf Cart Parts and Accessories - CartBarn.com...


In [99]:
for doc in docs:
    if 'owners.com' in doc.tags:
        display(doc.words)

['real_estat',
 'search',
 'buy_sell',
 'close',
 'readi_get_start',
 'call',
 'buysellsel',
 'homefre',
 'fsbo',
 'listingflat',
 'fee_ml',
 'full_servic',
 'agentwhat',
 'worth',
 'faqsmortgagesrequest',
 'pre',
 'approvalappli',
 'loanexplor',
 'loansmak',
 'inform',
 'decisionsspeak',
 'loan',
 'officermor',
 'serviceslearnhow',
 'worksadvicesign',
 'inregistermi',
 'accountown',
 'dashboardmi',
 'open',
 'housesemail',
 'settingsmessagesmi',
 'servicessign',
 'outour',
 'companyfaqscareerspressmor',
 'servicesemail',
 'uscallinstagramtwitterfacebookemili',
 'listingsmi',
 'searchessettingssign',
 'outsearchsearch',
 'homessearchcontinu',
 'recent',
 'search',
 'save',
 'searchessign',
 'inregisterprofil',
 'photomi',
 'accountown',
 'dashboardmi',
 'open',
 'housesemail',
 'settingsmessagesmi',
 'servicessign',
 'outsav',
 'email_updat',
 'search',
 'editlook',
 'like',
 'alreadi',
 'save',
 'search',
 'favoritestour',
 'listaddedbuysellsel',
 'homefre',
 'fsbo',
 'listingflat',
 